In [1]:
garbage_strings = [
    ("<>", 0),# empty garbage.
    ("<random characters>",17),# garbage containing random characters.
    ("<<<<>",3),# because the extra < are ignored.
    ("<{!>}>",2),# because the first > is canceled.
    ("<!!>",0),# because the second ! is canceled, allowing the > to terminate the garbage.
    ("<!!!>>",0),# because the second ! and the first > are canceled.
    ('<{o"i!a,<{i<a>',10),# which ends at the first >.
]

In [2]:
sample_strings = [
    ("{}", 1), # group.
    ("{{{}}}", 3), # groups.
    ("{{},{}}", 3), # groups.
    ("{{{},{},{{}}}}", 6), # groups.
    ("{<{},{},{{}}>}", 1), # group (which itself contains garbage).
    ("{<a>,<a>,<a>,<a>}", 1), # group.
    ("{{<a>},{<a>},{<a>},{<a>}}", 5), # groups.
    ("{{<!>},{<!>},{<!>},{<a>}}", 2), # groups (since all but the last > are canceled).
    ("{!!{!!}}", 2)
]

In [3]:
from anytree import Node, RenderTree

In [4]:
def count_groups(stream):
    root = Node('root')
    node_stack = [root]
    
    state = None
    cancel_next = False
    group_num = 0
    garbage_count = 0
    is_garbage = False
    for char in stream:
        #print(char, is_garbage)
        if cancel_next:
            cancel_next = False
            continue

        if char == '!':
            cancel_next = True
        elif char == '>':
            state = 'END_GARBAGE'
            is_garbage = False
        elif char == '{' and not is_garbage:
            state = 'START_GROUP'
            node_stack.append(Node('group%s'%group_num, parent=node_stack[-1]))
            group_num +=1
        elif char == '}' and not is_garbage:
            state = 'END_GROUP'
            node_stack.pop()
        elif char == '<' and not is_garbage:
            state = 'START_GARBAGE'
            is_garbage = True
        else:
            state = 'OTHER_CHAR'

        if is_garbage and not cancel_next and state not in ('START_GARBAGE', 'END_GARBAGE'):
            garbage_count += 1
        
    return root, garbage_count


In [5]:
def count_children(root):
    child_count = len(root.children)
    for child in root.children:
        child_count += count_children(child)
        
    return child_count

In [6]:
for sample_string in sample_strings:
    print(sample_string, count_children(count_groups(sample_string[0])[0]), sample_string[1])
    assert count_children(count_groups(sample_string[0])[0]) == sample_string[1]

for garbage_string in garbage_strings:
    print(garbage_string, count_groups(garbage_string[0])[1], garbage_string[1])
    assert count_groups(garbage_string[0])[1] == garbage_string[1]

('{}', 1) 1 1
('{{{}}}', 3) 3 3
('{{},{}}', 3) 3 3
('{{{},{},{{}}}}', 6) 6 6
('{<{},{},{{}}>}', 1) 1 1
('{<a>,<a>,<a>,<a>}', 1) 1 1
('{{<a>},{<a>},{<a>},{<a>}}', 5) 5 5
('{{<!>},{<!>},{<!>},{<a>}}', 2) 2 2
('{!!{!!}}', 2) 2 2
('<>', 0) 0 0
('<random characters>', 17) 17 17
('<<<<>', 3) 3 3
('<{!>}>', 2) 2 2
('<!!>', 0) 0 0
('<!!!>>', 0) 0 0
('<{o"i!a,<{i<a>', 10) 10 10


In [7]:
def get_score(root, depth, score):

    if len(root.children) == 0:
        return depth
    else:
        score = []
        for child in root.children:
            s = get_score(child, depth + 1, score)

            if type(s) == int:
                score = score + [s]
            else:
                score += s

    return depth, score

In [8]:
def nested_sum(l):
    total = 0
    for i in l:
        if isinstance(i, list):
            total += nested_sum(i)
        else:
            total += i
    return total

In [9]:
with open('data') as f:
    data = f.read()

In [10]:
nested_sum(get_score(count_groups(data)[0].children[0], 1, []))

21037

In [11]:
count_groups(data)[1]

9495